In [33]:
import random, sys, time

# Hash function.
#
# |key|: string
# Return value: a hash value
def calculate_hash(key):
    assert type(key) == str
    # Note: This is not a good hash function. Do you see why?
    hash = 0
    prime = 31
    for i in key:
        hash = hash * prime + ord(i) 
    return hash


# An item object that represents one key - value pair in the hash table.
class Item:
    # |key|: The key of the item. The key must be a string.
    # |value|: The value of the item.
    # |next|: The next item in the linked list. If this is the last item in the
    #         linked list, |next| is None.
    def __init__(self, key, value, next):
        assert type(key) == str
        self.key = key
        self.value = value
        self.next = next


# The main data structure of the hash table that stores key - value pairs.
# The key must be a string. The value can be any type.
#
# |self.bucket_size|: The bucket size.
# |self.buckets|: An array of the buckets. self.buckets[hash % self.bucket_size]
#                 stores a linked list of items whose hash value is |hash|.
# |self.item_count|: The total number of items in the hash table.
class HashTable:

    # Initialize the hash table.
    def __init__(self):
        # Set the initial bucket size to 97. A prime number is chosen to reduce
        # hash conflicts.
        self.bucket_size = 97
        self.buckets = [None] * self.bucket_size
        self.item_count = 0
                
    def reput(self, original_size, new_size):
        new_buckets = [None] * new_size
        new_count = 0
#         for i in range(original_size):
#             item = self.buckets[i]
#             if item is not None:
#                 new_bucket_index = calculate_hash(item.key) % new_size
#                 new_buckets[new_bucket_index] = item
        for i in range(original_size):
            item = self.buckets[i]
            while item is not None:
                new_bucket_index = calculate_hash(item.key) % new_size
                # 新しいバケットにアイテムを挿入
                new_item = Item(item.key, item.value, new_buckets[new_bucket_index])
                new_buckets[new_bucket_index] = new_item
                item = item.next
        self.buckets = new_buckets
        self.bucket_size = new_size

        
    # 要素数がテーブルサイズの70%を上回ったらテーブルサイズを2倍に拡張
    ## put時に検証
    # 要素数がテーブルサイズの30%を下回ったら、テーブルサイズを半分に縮小
    ## delete時に検証
    def rehash(self):
        # 要素数がテーブルサイズの70%を上回ったらテーブルサイズを2倍に拡張
        if self.item_count > self.bucket_size * 0.7:
            new_size = self.bucket_size * 2 + 1
            self.reput(self.bucket_size, new_size)
        # 要素数がテーブルサイズの30%を下回ったら、テーブルサイズを半分に縮小
        if self.item_count < self.bucket_size * 0.3 and  self.bucket_size > 97:
            new_size = max(97, self.bucket_size // 2 + 1)
            self.reput(self.bucket_size, new_size)
            

    # Put an item to the hash table. If the key already exists, the
    # corresponding value is updated to a new value.
    #
    # |key|: The key of the item.
    # |value|: The value of the item.
    # Return value: True if a new item is added. False if the key already exists
    #               and the value is updated.
    def put(self, key, value):
        assert type(key) == str
        self.check_size() # Note: Don't remove this code.
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]
        # 連結リストないのアイテムが存在する限り次のitemを見る
        ## 重複があればFalseを返す
        while item:
            if item.key == key:
                item.value = value
                return False
            item = item.next
        # self.buckets[bucket_index]の連結リストをnextに入れる
        new_item = Item(key, value, self.buckets[bucket_index])
        # self.bucket[bucket_index]の書き換え
        self.buckets[bucket_index] = new_item
        self.item_count += 1
#         print(str(self.item_count)+'put1')
        self.rehash()
#         print(str(self.item_count)+'put2')
        return True

    # Get an item from the hash table.
    #
    # |key|: The key.
    # Return value: If the item is found, (the value of the item, True) is
    #               returned. Otherwise, (None, False) is returned.
    def get(self, key):
        assert type(key) == str
        self.check_size() # Note: Don't remove this code.
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]
        while item:
            if item.key == key:
                return (item.value, True)
            item = item.next
        return (None, False)

    # Delete an item from the hash table.
    #
    # |key|: The key.
    # Return value: True if the item is found and deleted successfully. False
    #               otherwise.
    def delete(self, key):
        assert type(key) == str
        self.check_size()
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]
        prev_item = None
        while item:
            if item.key == key:
                # To do
                if prev_item is None:
                    next_item = item.next
                    self.buckets[bucket_index] = next_item
                else:
                    next_item = item.next
                    prev_item.next = next_item
                self.item_count -= 1
                self.rehash()
                return True
            prev_item = item
            item = item.next
        # Noneまで来たら
        return False

    # Return the total number of items in the hash table.
    def size(self):
        return self.item_count

    # Check that the hash table has a "reasonable" bucket size.
    # The bucket size is judged "reasonable" if it is smaller than 100 or
    # the buckets are 30% or more used.
    #
    # Note: Don't change this function.
    def check_size(self):
        assert (self.bucket_size < 100 or
                self.item_count >= self.bucket_size * 0.3)

# Test the functional behavior of the hash table.
def functional_test():
    hash_table = HashTable()

    assert hash_table.put("aaa", 1) == True
    assert hash_table.get("aaa") == (1, True)
    assert hash_table.size() == 1

    assert hash_table.put("bbb", 2) == True
    assert hash_table.put("ccc", 3) == True
    assert hash_table.put("ddd", 4) == True
    assert hash_table.get("aaa") == (1, True)
    assert hash_table.get("bbb") == (2, True)
    assert hash_table.get("ccc") == (3, True)
    assert hash_table.get("ddd") == (4, True)
    assert hash_table.get("a") == (None, False)
    assert hash_table.get("aa") == (None, False)
    assert hash_table.get("aaaa") == (None, False)
    assert hash_table.size() == 4

    assert hash_table.put("aaa", 11) == False
    assert hash_table.get("aaa") == (11, True)
    assert hash_table.size() == 4

    assert hash_table.delete("aaa") == True
    assert hash_table.get("aaa") == (None, False)
    assert hash_table.size() == 3

    assert hash_table.delete("a") == False
    assert hash_table.delete("aa") == False
    assert hash_table.delete("aaa") == False
    assert hash_table.delete("aaaa") == False

    assert hash_table.delete("ddd") == True
    assert hash_table.delete("ccc") == True
    assert hash_table.delete("bbb") == True
    assert hash_table.get("aaa") == (None, False)
    assert hash_table.get("bbb") == (None, False)
    assert hash_table.get("ccc") == (None, False)
    assert hash_table.get("ddd") == (None, False)
    assert hash_table.size() == 0

    assert hash_table.put("abc", 1) == True
    assert hash_table.put("acb", 2) == True
    assert hash_table.put("bac", 3) == True
    assert hash_table.put("bca", 4) == True
    assert hash_table.put("cab", 5) == True
    assert hash_table.put("cba", 6) == True
    assert hash_table.get("abc") == (1, True)
    assert hash_table.get("acb") == (2, True)
    assert hash_table.get("bac") == (3, True)
    assert hash_table.get("bca") == (4, True)
    assert hash_table.get("cab") == (5, True)
    assert hash_table.get("cba") == (6, True)
    assert hash_table.size() == 6

    assert hash_table.delete("abc") == True
    assert hash_table.delete("cba") == True
    assert hash_table.delete("bac") == True
    assert hash_table.delete("bca") == True
    assert hash_table.delete("acb") == True
    assert hash_table.delete("cab") == True
    assert hash_table.size() == 0
    print("Functional tests passed!")


# Test the performance of the hash table.
#
# Your goal is to make the hash table work with mostly O(1).
# If the hash table works with mostly O(1), the execution time of each iteration
# should not depend on the number of items in the hash table. To achieve the
# goal, you will need to 1) implement rehashing (Hint: expand / shrink the hash
# table when the number of items in the hash table hits some threshold) and
# 2) tweak the hash function (Hint: think about ways to reduce hash conflicts).
def performance_test():
    hash_table = HashTable()

    for iteration in range(100):
        begin = time.time()
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.put(str(rand), str(rand))
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.get(str(rand))
        end = time.time()
        print("%d %.6f" % (iteration, end - begin))

    for iteration in range(100):
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.delete(str(rand))

    print(hash_table.size())
    assert hash_table.size() == 0
    print("Performance tests passed!")


if __name__ == "__main__":
    functional_test()
    performance_test()

Functional tests passed!
0 0.100941
1 0.201465
2 0.111336
3 0.150338
4 0.113876
5 0.060905
6 0.062104
7 0.320182
8 0.056662
9 0.058136
10 0.058686
11 0.135372
12 0.061031
13 0.072880
14 0.540540
15 0.146770
16 0.061155
17 0.059331
18 0.058553
19 0.057377
20 0.058141
21 0.057756
22 0.073610
23 0.181498
24 0.062532
25 0.060185
26 0.061356
27 0.067377
28 1.104646
29 0.058547
30 0.059104
31 0.058894
32 0.062327
33 0.059218
34 0.059658
35 0.057799
36 0.059407
37 0.059294
38 0.057503
39 0.219057
40 0.059996
41 0.059077
42 0.060606
43 0.065168
44 0.063246
45 0.062313
46 0.060327
47 0.061630
48 0.063528
49 0.070324
50 0.065536
51 0.062280
52 0.317795
53 0.061685
54 0.060754
55 0.059866
56 2.108336
57 0.062119
58 0.059464
59 0.058995
60 0.059074
61 0.061212
62 0.058907
63 0.059007
64 0.059478
65 0.061321
66 0.057793
67 0.057274
68 0.057719
69 0.059024
70 0.308343
71 0.060951
72 0.061630
73 0.068908
74 0.063635
75 0.066365
76 0.063490
77 0.074696
78 0.063412
79 0.069128
80 0.065364
81 0.067415
8